In [1]:
pip install pandas numpy transformers scikit-learn tqdm

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import os

In [12]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("已释放 PyTorch CUDA 缓存")

seed_everything(seed=2025)

已释放 PyTorch CUDA 缓存


In [8]:
data = pd.read_csv('dataset/train_all.csv')

In [13]:
def get_label(data):
    unique_labels = sorted(data['类别'].unique())
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    id_to_label = {idx: label for label, idx in label_to_id.items()}
    data['label'] = data['类别'].map(label_to_id)

    return label_to_id, id_to_label, data

label_to_id,id_to_label, data = get_label(data)

In [14]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
class Lookahead(Optimizer):
    def __init__(self, optimizer, k=5, alpha=0.5):
        self.optimizer = optimizer
        self.k = k
        self.alpha = alpha
        self.param_groups = self.optimizer.param_groups
        self.state = defaultdict(dict)
        self.fast_state = self.optimizer.state
        for group in self.param_groups:
            group["counter"] = 0

    def update(self, group):
        for fast in group["params"]:
            param_state = self.state[fast]
            if "slow_param" not in param_state:
                param_state["slow_param"] = torch.zeros_like(fast.data)
                param_state["slow_param"].copy_(fast.data)
            slow = param_state["slow_param"]
            slow += (fast.data - slow) * self.alpha
            fast.data.copy_(slow)

    def update_lookahead(self):
        for group in self.param_groups:
            self.update(group)

    def step(self, closure=None):
        loss = self.optimizer.step(closure)
        for group in self.param_groups:
            if group["counter"] == 0:
                self.update(group)
            group["counter"] += 1
            if group["counter"] >= self.k:
                group["counter"] = 0
        return loss

    def state_dict(self):
        fast_state_dict = self.optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict["state"]
        param_groups = fast_state_dict["param_groups"]
        return {
            "fast_state": fast_state,
            "slow_state": slow_state,
            "param_groups": param_groups,
        }

    def load_state_dict(self, state_dict):
        slow_state_dict = {
            "state": state_dict["slow_state"],
            "param_groups": state_dict["param_groups"],
        }
        fast_state_dict = {
            "state": state_dict["fast_state"],
            "param_groups": state_dict["param_groups"],
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.optimizer.load_state_dict(fast_state_dict)
        self.fast_state = self.optimizer.state

    def add_param_group(self, param_group):
        param_group["counter"] = 0
        self.optimizer.add_param_group(param_group)

In [21]:
# class BertForSequenceClassification(nn.Module):
#     def __init__(self, bert_model_name, num_labels=10, dropout=0.1):
#         super(BertForSequenceClassification, self).__init__()
#         self.num_labels = num_labels
#         self.bert = BertModel.from_pretrained(bert_model_name)
#         self.dropout = nn.Dropout(dropout)
#         self.hidden_size = self.bert.config.hidden_size
#         self.classifier = nn.Linear(self.hidden_size, num_labels)

#     def forward(self, input_ids, attention_mask, token_type_ids):
#         outputs = self.bert(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             return_dict=False
#         )
#         pooled_output = outputs[1]
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)

#         return logits

class BertForSequenceClassification(nn.Module):
    def __init__(self, bert_model_name, num_labels=10, dropout=0.1):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained(bert_model_name, output_hidden_states=True)
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(self.hidden_size * 5, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False
        )

        last_hidden_state = outputs[0]
        pooled_output = outputs[1]
        hidden_states = outputs[2]

        cls_list = []
        for i in range(-1, -6, -1):
            cls_output = hidden_states[i][:, 0, :]
            cls_list.append(cls_output)

        last_hidden = torch.cat(cls_list, dim=1)
        last_hidden = self.dropout(last_hidden)
        logits = self.classifier(last_hidden)

        return logits

In [22]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon=1., emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [23]:
def train_model_with_FGM(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, device, num_epochs,save_path):

    best_macro_f1 = 0.0
    for epoch in range(num_epochs):

        model.train()
        total_loss = 0
        progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}")

        fgm = FGM(model)
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits, labels)
            loss.backward(retain_graph=True)

            fgm.attack()
            logits_adv = model(input_ids, attention_mask, token_type_ids)
            loss_adv = criterion(logits_adv, labels)
            loss_adv.backward(retain_graph=True)
            fgm.restore()

            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})

        avg_train_loss = total_loss / len(train_dataloader)

        val_loss, val_macro_f1 = evaluate_model(model, val_dataloader, criterion, device)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Val Macro F1: {val_macro_f1:.4f}")
        print("-" * 50)

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved to {save_path} with Val Macro F1: {val_macro_f1:.4f}")

    print(f"Training completed. Best Macro F1: {best_macro_f1:.4f}")

In [24]:
class PGD():
    def __init__(self, model):
        self.model = model
        self.emb_backup = {}
        self.grad_backup = {}

    def attack(self, epsilon=1., alpha=0.3, emb_name='word_embeddings', is_first_attack=False):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                if is_first_attack:
                    self.emb_backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0 and not torch.isnan(norm):
                    r_at = alpha * param.grad / norm
                    param.data.add_(r_at)
                    param.data = self.project(name, param.data, epsilon)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.emb_backup
                param.data = self.emb_backup[name]
        self.emb_backup = {}

    def project(self, param_name, param_data, epsilon):
        r = param_data - self.emb_backup[param_name]
        if torch.norm(r) > epsilon:
            r = epsilon * r / torch.norm(r)
        return self.emb_backup[param_name] + r

    def backup_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.grad_backup[name] = param.grad.clone()

    def restore_grad(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.grad = self.grad_backup[name]

In [25]:
def train_model_with_PGD(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, device, num_epochs, save_path):

    best_macro_f1 = 0.0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}")

        pgd = PGD(model)
        K = 3
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits, labels)
            loss.backward()

            # 对抗训练
            pgd.backup_grad()
            for t in range(K):
                pgd.attack(is_first_attack=(t==0))
                if t != K-1:
                    model.zero_grad()
                else:
                    pgd.restore_grad()
                logits_adv = model(input_ids, attention_mask, token_type_ids)
                loss_adv = criterion(logits_adv, labels)
                loss_adv.backward()
            pgd.restore()

            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})

        avg_train_loss = total_loss / len(train_dataloader)

        val_loss, val_macro_f1 = evaluate_model(model, val_dataloader, criterion, device)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Val Macro F1: {val_macro_f1:.4f}")
        print("-" * 50)

        if val_macro_f1 > best_macro_f1:
            best_macro_f1 = val_macro_f1
            torch.save(model.state_dict(), save_path)
            print(f"Best model saved to {save_path} with Val Macro F1: {val_macro_f1:.4f}")

    print(f"Training completed. Best Macro F1: {best_macro_f1:.4f}")

In [26]:
def evaluate_model(model, dataloader, criterion, device):
    """
    评估模型，返回损失、准确率和 Macro F1-Score
    """
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_preds = []
    all_labels = []

    progress_bar = tqdm(dataloader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.size(0)

            progress_bar.set_postfix({'batch_loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(dataloader)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')

    return avg_loss, macro_f1

In [28]:
MODEL_NAME = '/workspace/models/chinese-bert-wwm'
MAX_LENGTH = 512
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 0.1
NUM_EPOCHS = 10
DROPOUT = 0.2
NUM_LABELS = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification(bert_model_name=MODEL_NAME,num_labels=NUM_LABELS,dropout=DROPOUT).to(device)

In [29]:
train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['label'])

train_data_text = train_data['文本'].to_list()
train_data_label = train_data['label'].to_list()

val_data_text = val_data['文本'].to_list()
val_data_label = val_data['label'].to_list()

train_dataset = TextClassificationDataset(train_data_text, train_data_label, tokenizer, MAX_LENGTH)
val_dataset = TextClassificationDataset(val_data_text, val_data_label, tokenizer, MAX_LENGTH)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [30]:
def softmax_with_temperature(logits, temperature=1.0):
    return np.exp((logits - np.max(logits)) / temperature) / np.sum(np.exp((logits - np.max(logits)) / temperature))

In [40]:
compute_class_weight('balanced', classes=np.array(list(id_to_label.keys())), y=train_data_label)

array([ 0.80804829, 80.32      ,  3.23349436, 10.40414508,  1.57490196,
       35.22807018,  0.3543321 ,  2.19213974,  0.55933148,  0.38240335])

array([ 2.72973723, 65.67270703,  3.00784296,  4.00702561,  2.81476703,
       10.81578053,  2.68064304,  2.88512726,  2.70271461,  2.68365469])

In [39]:
# CrossEntropyLoss
# criterion = nn.CrossEntropyLoss()

# class_weights CrossEntropyLoss
class_weights = torch.tensor(softmax_with_temperature(compute_class_weight('balanced', classes=np.array(list(id_to_label.keys())), y=train_data_label), temperature=30.0), dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
total_steps = len(train_dataloader) * NUM_EPOCHS
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=int(0.05 * total_steps), num_training_steps=total_steps)

In [ ]:
save_path = 'model_path/best_model_with_FGM_macro_f1.pth'
train_model_with_FGM(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, device, num_epochs=NUM_EPOCHS, save_path=save_path)

Training Epoch 1: 100%|██████████| 2510/2510 [37:02<00:00,  1.13it/s, loss=0.513] 


Epoch 1/10
  Val Macro F1: 0.5796
--------------------------------------------------
Best model saved to ./model_path/best_model_with_FGM_macro_f1.pth with Val Macro F1: 0.5796


Training Epoch 2: 100%|██████████| 2510/2510 [37:01<00:00,  1.13it/s, loss=0.534]  


Epoch 2/10
  Val Macro F1: 0.6128
--------------------------------------------------
Best model saved to ./model_path/best_model_with_FGM_macro_f1.pth with Val Macro F1: 0.6128


Training Epoch 3: 100%|██████████| 2510/2510 [37:11<00:00,  1.12it/s, loss=0.0666] 


Epoch 3/10
  Val Macro F1: 0.6388
--------------------------------------------------
Best model saved to ./model_path/best_model_with_FGM_macro_f1.pth with Val Macro F1: 0.6388


Training Epoch 4:  82%|████████▏ | 2052/2510 [30:20<06:46,  1.13it/s, loss=0.0859] 

In [ ]:
def predict_single_text(model, text, tokenizer, label_to_id, id_to_label, device, max_length=512):

    model.eval()

    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    token_type_ids = encoding['token_type_ids'].to(device)
    with torch.no_grad():
        logits = model(input_ids, attention_mask, token_type_ids)

    probabilities = F.softmax(logits, dim=1)
    predicted_id = torch.argmax(logits, dim=1).item()
    predicted_label = id_to_label[predicted_id]

    return predicted_id,predicted_label

In [ ]:
loaded_model = BertForSequenceClassification(bert_model_name=MODEL_NAME,num_labels=NUM_LABELS).to(device)
loaded_model.load_state_dict(torch.load(save_path, map_location=device))
loaded_model.to(device)
loaded_model.eval()

In [ ]:
data_test = pd.read_csv('dataset/test_text.csv')
data_test_text = data_test['文本'].to_list()

pred_list = []
for text in tqdm(data_test_text):
    pred_id, pred_label = predict_single_text(
        model=loaded_model,
        text=text,
        tokenizer=tokenizer,
        label_to_id=label_to_id,
        id_to_label=id_to_label,
        device=device,
        max_length=MAX_LENGTH
    )
    pred_list.append(pred_label)

submit = pd.read_csv('dataset/example.csv')
submit['类别'] = pred_list
submit.to_csv('dataset/submit.csv')